<center><img src='https://raw.githubusercontent.com/Jangrae/img/master/title.png' width=500/></center>


# **이변량 분석 ② - 범주형  → 수치형**


- 시각화와 수치화 방법으로 범주형 변수와 수치형 변수 간 관계를 분석합니다.

<img src='https://raw.githubusercontent.com/Jangrae/img/master/two_var_02.png' width=600 align='left'/>

<img src='https://raw.githubusercontent.com/Jangrae/img/master/analysis2.png' width=620 align='left'/>

## **1. 환경준비**

- 사용할 라이브러리와 분석 대상 데이터를 읽어옵니다.

In [ ]:
import numpy as np
import pandas as pd
import random as rd
import matplotlib.pyplot as plt
import seaborn as sns
import scipy.stats as spst
import warnings

warnings.filterwarnings(action='ignore')
%config InlineBackend.figure_format = 'retina'
plt.rcParams['figure.figsize'] = (6, 4)

In [ ]:
# Titanic 데이터
path = 'https://raw.githubusercontent.com/Jangrae/csv/master/titanic.csv'
titanic = pd.read_csv(path)
titanic.head(10)

In [ ]:
# Insurance 데이터
path = 'https://raw.githubusercontent.com/Jangrae/csv/master/insurance.csv'
insurance = pd.read_csv(path)
insurance.head(10)

## **2. 시각화**

- **Seaborn**의 **barplot()** 함수는 범줏값 간의 **평균을 비교**해 표시합니다.
- **Seaborn**의 **boxplot()** 함수로 범줏값 간의 **값 분포를 비교**할 수 있습니다.

In [ ]:
sns.barplot(x='Survived', y='Age', data=titanic)
plt.show()

In [ ]:
sns.boxplot(x='Survived', y='Age', data=titanic)
plt.show()

- **질문:** 생존 여부(Survived) 별 나이(Age) 평균에 차이가 있나요?

In [ ]:
#

In [ ]:
plt.figure(figsize=(8, 3))
plt.subplot(1, 2, 1)
sns.barplot(x='Pclass', y='Age', data=titanic)
plt.subplot(1, 2, 2)
sns.boxplot(x='Pclass', y='Age', data=titanic)
plt.tight_layout()
plt.show()

- **질문:**  객실 등급(Pclass)별 나이(Age) 평균에 차이가 있나요?

In [ ]:
#

<img src='https://raw.githubusercontent.com/jangrae/img/master/practice_20.png' width=120 align="left"/>

insurance 데이터프레임의 다음 변수들 간의 관계를 barplot(), boxplot() 함수로 확인하세요.

**1️⃣ sex → charges**

- **질문:**  남녀(sex)별 의료비(charges) 평균에 차이가 있나요?

In [ ]:
#

**2️⃣ smoker → charges**

- **질문:**  흡연 여부(smoker)별 의료비(charges) 평균에 차이가 있나요?

In [ ]:
#

**3️⃣ region → charges**

- **질문:**  지역(region)별 의료비(charges) 평균에 차이가 있나요?

In [ ]:
#

<img src='https://raw.githubusercontent.com/jangrae/img/master/practice_20e.png' width=45 align="left"/>

## **3. 수치화**

- 이제는 시각화 대신 수치화를 통해 비교해 봅니다.
- 다음 두 가지 방법으로 수치화해 확인합니다.
    - t-검정(두 범주)
    - ANOVA(세 범주 이상)

### **3.1. t-검정**

- 두 집단의 평균값이 서로 유의하게 다른지를 검정하는 통계적인 방법입니다.
- 데이터에 결측치가 있으면 계산이 안되니, notnull() 등으로 결측치를 제외한 데이터를 사용해야 합니다.
- **scipy.stats** 라이브러리의 **ttest_ind()** 함수로 t-검정을 수행합니다



**📌 t-statistics, t-통계량**
- 두 평균의 차이를 표준오차로 나눈 값입니다.
- 기본적으로는 두 평균의 차이로 이해해도 좋습니다.
- 우리의 가설(대립가설)은 차이가 있다는 것이므로, t-통계량 값이 크던지 작던지 하기를 바랍니다.
- 보통, **t-통계량 값이 -2보다 작거나, 2보다 크면 차이가 있다고 봅니다.**

**📌 p-value**

- t-검정에서 p-value는 유의확률(Significance Probability)로도 불립니다.
- 귀무가설(Null Hypothesis)을 검정할 때 사용되는 값입니다.
- p-value는 귀무가설이 맞을 때 표본에서 계산된 통계량이 나올 확률을 나타냅니다.
- 따라서, p-value가 작을수록 귀무가설이 틀릴 확률이 높아지므로, 귀무가설을 기각하게 됩니다.
- **p-value 값이 작으면 통계적으로 유의미한 차이가 있다는 것을 의미한다고 보면 됩니다**

**📌 표준오차**

- 모집단의 평균을 구하고 싶지만, 모집단이 너무 커서 구할 수 없습니다.
- 그래서 모집단에서 표본(Sample)을 뽑아 그 표본의 평균을 구합니다.
- 계속해서 표본을 뽑아 그 표본의 평균을 구합니다.
- 이 표본들의 평균의 평균이 모집단의 평균입니다.
- 그리고 이 표본들의 평균의 표준편차를 **표준오차**라고 부릅니다.
- 즉 추정값인 표본 평균들과 참값인 모평균과의 표준적인 차이라고 할 수 있습니다.

- 우선 두 개의 모집단을 만듭니다.
    - pop1: 평균이 160, 표준편차가 10인 100,000개의 실수
    - pop2: 평균이 170, 표준편차가 10인 100,000개의 실수
    

In [ ]:
pop1 = [round(rd.normalvariate(160, 10), 1) for i in range(100000)]
pop2 = [round(rd.normalvariate(170, 10), 1) for i in range(100000)]

# 확인
print('* pop1:', pop1[:10])
print('* pop2:', pop2[:10])

- 우리는 모집단을 알 수 없지만 그래도 살짝 살펴봅니다.

In [ ]:
plt.figure(figsize=(8, 5))
sns.kdeplot(pop1)
sns.kdeplot(pop2)
plt.show()

- 이제 두 모집단으로 부터 표본을 200개씩 추출한 후 t-검정을 수행합니다.
- 먼저 모집단 pop1으로 부터 두 개의 샘플을 추출한 후 t-검정을  수행합니다.

In [ ]:
s1 = rd.sample(pop1, 200)
s2 = rd.sample(pop1, 200)
t_test = spst.ttest_ind(s1, s2)

print('* t-statistic:', t_test[0])
print('* p-value:', t_test[1])

- **질문:**  p-value의 의미는?

In [ ]:
#

- 이번에는 서로 다른 모집단으로 부터 각각 샘플링한 후 t-검정을 수행합니다.

In [ ]:
s1 = rd.sample(pop1, 200)
s2 = rd.sample(pop2, 200)
t_test = spst.ttest_ind(s1, s2)

print('* t-statistic:', t_test[0])
print('* p-value:', t_test[1])

- **질문:**   p-value의 의미는?

In [ ]:
#

- 이번엔 titanic 데이터프레임으로 시도해 봅니다.
- **Survived → Age**: 생존 여부 별로 나이에 차이가 있을것이다.
- 먼저 결측치가 있는지 확인해 봅니다.

In [ ]:
titanic.isna().sum()

- Age 변수에 결측치가 없는 행만 가져와 다음 두 변수를 선언합니다.
    - died: 사망자들의 나이
    - survived: 생존자들의 나이
- 이 두 변수가 가지는 나이(Age)에 차이가 있을지 궁금합니다.

In [ ]:
temp = titanic.loc[titanic['Age'].notnull()]
died = temp.loc[temp['Survived']==0, 'Age']
survived = temp.loc[temp['Survived']==1, 'Age']

t_test = spst.ttest_ind(died, survived)

print('* t-statistic:', t_test[0])
print('* p-value:', t_test[1])

- **질문:**   p-value의 의미는?

In [ ]:
#
#

- **Sex → Fare**: 성별에 따라 운임에 차이가 있을 것이다.

In [ ]:
male = titanic.loc[titanic['Sex']=='male', 'Fare']
female = titanic.loc[titanic['Sex']=='female', 'Fare']

t_test = spst.ttest_ind(male, female)

print('* t-statistic:', t_test[0])
print('* p-value:', t_test[1])

- **질문:**   p-value의 의미는?

In [ ]:
#

<img src='https://raw.githubusercontent.com/jangrae/img/master/practice_20.png' width=120 align="left"/>

insurance 데이터프레임의 다음 변수들 간의 관계를 t-통계량과 p-value를 사용해 확인하세요.

**1️⃣ sex → charges**

- **질문:**  남녀(sex)별 의료비(charges) 평균에 차이가 있나요?

In [ ]:
#

**2️⃣ smoker → charges**

- **질문:**  흡연 여부(smoker)별 의료비(charges) 평균에 차이가 있나요?

In [ ]:
#

<img src='https://raw.githubusercontent.com/jangrae/img/master/practice_20e.png' width=45 align="left"/>

### **3.2. 분산분석**

- 세 개 이상의 집단 간 차이는 분산분석(ANOVA)으로 비교합니다.
- ANOVA: **AN**alysis **O**f **VA**riance
- 여기서 기준은 전체 평균 입니다.
- **scipy.stats** 라이브러리의 **f_oneway()** 함수로 분산분석을 수행합니다.

**📌 f-statistics, f-통계량, f-value**

- 두 개 이상의 집단 간 분산의 비율을 검정하는 데 사용되는 통계량입니다.
- 분산분석(ANOVA)에서 사용되며, 각 집단 내에서의 분산과 각 집단 간의 분산의 비율을 계산합니다.
- $f-통계량=\frac{집단간 분산}{집단내 분산}  $
- 이 계산 결과로 각 집단이 모집단으로부터 동일한 분산을 가지는지를 검정합니다.
- 분산의 비율이 작을수록 1에 가까운 값을 가집니다.
- 분산의 비율이 클수록 f-통계량 값이 커집니다.
- f-통계량 값이 크다는 것은 각 집단 간 차이가 통계적으로 유의미하다는 것을 나타냅니다.
- 보통, **f-통계량 값이 2~3 이상이면 차이가 있다고 봅니다.**

- **Pclass → Age**: 객실 등급별로 나이에 차이가 있을 것이다.

In [ ]:
print('* 전체평균: {}'.format(titanic['Age'].mean()))

sns.barplot(x='Pclass', y='Age', data=titanic)
plt.show()

- Pclass 값을 기준으로 p1 ~ p3 세 집단을 만듭니다.
- 결측치가 없는 데이터만을 대상으로 합니다.

In [ ]:
temp = titanic.loc[titanic['Age'].notnull()]
p1 = temp.loc[titanic['Pclass']==1, 'Age']
p2 = temp.loc[titanic['Pclass']==2, 'Age']
p3 = temp.loc[titanic['Pclass']==3, 'Age']
anova = spst.f_oneway(p1, p2, p3)
print('* f-statistic:', anova[0])
print('* p-value:', anova[1])

- **Embarked → Fare**: 승선 지역별로 운임에 차이가 있을 것이다.

In [ ]:
print('* 전체평균: {}'.format(titanic['Fare'].mean()))

sns.barplot(x='Embarked', y='Fare', data=titanic)
plt.show()

<img src='https://raw.githubusercontent.com/jangrae/img/master/practice_20.png' width=120 align="left"/>

insurance 데이터프레임의 다음 변수 간의 관계를 f-통계량과 p-value를 사용해 확인하세요.

**1️⃣ region → charges**

- **질문:**  지역(region)별 의료비(charges) 평균에 차이가 있나요?

In [ ]:
#

<img src='https://raw.githubusercontent.com/jangrae/img/master/practice_20e.png' width=45 align="left"/>